In [11]:
import numpy as np
import math as m
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import os
import pandas as pd

In [18]:
os.chdir('drawdown')

'/users/1/souce015/BearSpring/BearSpringUMN/drawdown'

In [65]:
df = pd.read_csv('galena_drawdown.csv')
df = df.dropna(how='all', axis=1)  # Remove completely empty columns
df = df.dropna(how='all', axis=0)  # Remove completely empty rows
df.iloc[:,0] = df.iloc[:,0] * 1440
print(df.head())  # Check the cleaned data

df.to_csv('cleaned_galena_drawdown.csv', index=False)

   1.15278E-05  0.625
0       0.0250  0.866
1       0.0333  1.138
2       0.0416  1.411
3       0.0500  1.651
4       0.0583  1.859


In [66]:
#From the pyTheis paper, 2021
def wufunc(r,S,T,t):
    u = (r**2)*S/(4*T*t)
    Wu=-0.5772-np.log(u)+u
    ntrm = 30
    for i in range (2, ntrm+1):
        sign = (-1)**(i-1)
        factval = float(m.factorial(i))
        Wu = Wu + sign*(u**i)/(i*factval)
    return(Wu)
    
def myfunc(tt, T, S):
    pi = 3.14
    Q = 1199.22 #m3/d
    r = 251.155 #m
    nrow = len(tt)
    n = nrow
    drawdown=np.zeros((n),float)
    for i in range (0,n):
        Wu_val=wufunc(r,S,T,tt[i])
        drawdown[i] = Q*Wu_val/(4*pi*T)
    #End loop i
    return (drawdown)

def main():
    indata = np.loadtxt("cleaned_galena_drawdown.csv", delimiter=",")
    Times = np.copy(indata[:,0]) # Time from first column
    sobs = np.copy(indata[:,1]) # Drawdown from second column
    init_vals = [1, 0.001] # for [T & S values]
    best_vals, covar = curve_fit(myfunc, Times, sobs, p0=init_vals, bounds=([0.01, 0.000001], [100000, 0.1]), method = 'trf')
    stdevs = np.sqrt(np.diag(covar))
    plt.xscale('log')
    plt.yscale('log')
    print ('Best valus (T, S)')
    print (best_vals)
    print ('Covariance')
    print (covar)
    print ('standard deviation')
    print (stdevs)
    T = best_vals [0]
    S = best_vals [1]
    smodel=myfunc(Times,T,S)
    plt.plot(Times,smodel)
    plt.plot(Times, sobs, 'bo')
    plt.xlabel("Time (day)")
    plt.ylabel("Drawdown (m)")
    plt.show()

In [67]:
main()

/users/1/souce015/.conda/envs/modflow6/lib/python3.10/site-packages/scipy/optimize/_lsq/trf.py:238: RuntimeWarning: overflow encountered in multiply
  g_norm = norm(g * v, ord=np.inf)
/users/1/souce015/.conda/envs/modflow6/lib/python3.10/site-packages/scipy/optimize/_lsq/trf.py:263: RuntimeWarning: overflow encountered in multiply
  g_norm = norm(g * v, ord=np.inf)
/users/1/souce015/.conda/envs/modflow6/lib/python3.10/site-packages/scipy/optimize/_lsq/trf.py:294: RuntimeWarning: overflow encountered in multiply
  g_h = d * g


ValueError: array must not contain infs or NaNs